<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/bert_replace/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=e0741274fe9e03e5c7838fda83b4aadbd89a3e3b70a3a6e06d4abbe77e986de3
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=0dbb4ec37c2153bf23eabdeda051d7a0ab95e45c5edb7f5565a1f55f20a1eff0
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=6fd9a47633728cfabb389ab3af661835313741b81ac924c103596dc229f36cac
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=7b02a979fd5c9a678935236af43cfff401706be84274f02e3a8e864e7a5cb418
  Stor

In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 121 (delta 56), reused 84 (delta 27), pack-reused 0
Receiving objects: 100% (121/121), 5.76 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/bachelor_design


In [17]:
!git checkout bert_replace
!git pull

Already on 'bert_replace'
Your branch is up to date with 'origin/bert_replace'.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/supplient/bachelor_design
   409b6b4..f0a598b  bert_replace -> origin/bert_replace
Updating 409b6b4..f0a598b
Fast-forward
 util.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


# Mount GDriver
Perpare Colab environment.

In [4]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


# Prepare Input

In [5]:
import preprocess
import config

Using TensorFlow backend.


In [0]:
char_seqs, tag_seqs = preprocess.load_file(
    addh + config.DATA_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

In [0]:
import pickle
with open(addh + config.TAG_VOCAB_PATH, "wb") as fd:
    pickle.dump(tag_vocab, fd)

In [8]:
tag_vocab

{'': 0,
 'B-EI': 2,
 'B-EIF': 10,
 'B-EO': 8,
 'B-EQ': 6,
 'B-ILF': 4,
 'I-EI': 3,
 'I-EIF': 11,
 'I-EO': 9,
 'I-EQ': 7,
 'I-ILF': 5,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [10]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(len(tag_vocab), activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [12]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

(1, 512, 12)

In [13]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [14]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512, 12)      9228        model_2[1][0]              

# Fine-tuning

In [15]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

In [0]:
import util
epoch_callback = util.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_LOG_PATH,
    1
    )

In [24]:
model.fit(
    x=[token_id_seqs, segment_seqs],
    y=[tag_id_seqs], 
    batch_size=8, 
    epochs=5,
    verbose=2,
    validation_split=0.1,
    callbacks=[epoch_callback]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7005 samples, validate on 779 samples
Epoch 1/5
 - 595s - loss: 0.5406 - acc: 0.8316 - val_loss: 0.4211 - val_acc: 0.8587
Epoch 2/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7005 samples, validate on 779 samples
Epoch 1/5
 - 595s - loss: 0.5406 - acc: 0.8316 - val_loss: 0.4211 - val_acc: 0.8587
Epoch 2/5
 - 578s - loss: 0.3835 - acc: 0.8670 - val_loss: 0.3797 - val_acc: 0.8786
 - 578s - loss: 0.3835 - acc: 0.8670 - val_loss: 0.3797 - val_acc: 0.8786
Epoch 3/5
Epoch 3/5
 - 578s - loss: 0.3107 - acc: 0.8909 - val_loss: 0.3553 - val_acc: 0.8806
 - 578s - loss: 0.3107 - acc: 0.8909 - val_loss: 0.3553 - val_acc: 0.8806
Epoch 4/5
Epoch 4/5
 - 578s - loss: 0.2573 - acc: 0.9094 - val_loss: 0.3656 - val_acc: 0.8862
 - 578s - loss: 0.2573 - acc: 0.9094 - val_loss: 0.3656 - val_acc: 0.8862
Epoch 5/5
Epoch 5/5
 - 578s - loss: 0.2102

In [0]:
model.save(addh + config.MODEL_PATH)